In [0]:
# Importing the libraries
import numpy as np
import tensorflow as tf
import os

In [0]:
# Loading the text file containing shakespere's text and converting to numbers
if 'shakespeare_input.txt' not in os.listdir():
  !wget https://cs.stanford.edu/people/karpathy/char-rnn/shakespeare_input.txt
with open('shakespeare_input.txt', 'r') as f:
    text=f.read()
vocab = sorted(set(text))
vocab_to_int = {c: i for i, c in enumerate(vocab)}
int_to_vocab = dict(enumerate(vocab))
encoded = np.array([vocab_to_int[c] for c in text], dtype=np.int32)

In [4]:
# Original text
print(text[:100])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [5]:
# Encoded text
print(encoded[:100])

[18 49 58 59 60  1 15 49 60 49 66 45 54 10  0 14 45 46 55 58 45  1 63 45
  1 56 58 55 43 45 45 44  1 41 54 65  1 46 61 58 60 48 45 58  6  1 48 45
 41 58  1 53 45  1 59 56 45 41 51  8  0  0 13 52 52 10  0 31 56 45 41 51
  6  1 59 56 45 41 51  8  0  0 18 49 58 59 60  1 15 49 60 49 66 45 54 10
  0 37 55 61]


In [6]:
# size of vocal
vocab_size = len(vocab)
print(vocab_size)

67


In [0]:
# function for getting batches
def get_batches(arr, batch_size, n_steps):
    '''Create a generator that returns batches of size
       batch_size x n_steps from arr.
    '''
    # Get the number of characters per batch and number of batches we can make
    characters_per_batch = batch_size * n_steps
    n_batches = len(arr) // characters_per_batch
    
    # Keep only enough characters to make full batches
    arr = arr[: n_batches * characters_per_batch]
    
    # Reshape into batch_size rows
    arr = arr.reshape((batch_size, -1))
    
    for n in range(0, arr.shape[1], n_steps):
        # The features
        x = arr[:, n : n + n_steps]
        # The targets
        y = np.zeros_like(x)
        y[:, :-1], y[:, -1] = x[:, 1:], x[:, 0]
        yield x, y

In [0]:
batches = get_batches(encoded, 10, 50)
x, y = next(batches)

In [40]:
print('x\n', x[:10, :10])
print('\ny\n', y[:10, :10])

x
 [[18 49 58 59 60  1 15 49 60 49]
 [45 59 45  6  1 63 48 55  1 55]
 [52  1 42 45  1 48 41 54 47 45]
 [ 1 43 41 60  1 49 59  1 47 58]
 [48  1 44 49 44  1 57 61 41 51]
 [54 44 45 58  1 48 55 54 55 61]
 [10  0 25 65  1 63 55 58 60 48]
 [ 0 32 49 52 52  1 48 45  1 48]
 [41 44  1 55 46  1 48 49 53  8]
 [46  1 65 55 61  1 63 49 52 52]]

y
 [[49 58 59 60  1 15 49 60 49 66]
 [59 45  6  1 63 48 55  1 55 46]
 [ 1 42 45  1 48 41 54 47 45 44]
 [43 41 60  1 49 59  1 47 58 41]
 [ 1 44 49 44  1 57 61 41 51 45]
 [44 45 58  1 48 55 54 55 61 58]
 [ 0 25 65  1 63 55 58 60 48 65]
 [32 49 52 52  1 48 45  1 48 41]
 [44  1 55 46  1 48 49 53  8  0]
 [ 1 65 55 61  1 63 49 52 52  1]]


In [0]:
# Creating placeholders for input, target and keep_prob
def build_inputs(batch_size, num_steps):
    ''' Define placeholders for inputs, targets, and dropout 
    
    '''
    # Declare placeholders we'll feed into the graph
    inputs = tf.placeholder(tf.int32, [batch_size, num_steps], name='inputs')
    targets = tf.placeholder(tf.int32, [batch_size, num_steps], name='targets')
    
    # Keep probability placeholder for drop out layers
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')
    
    return inputs, targets, keep_prob

In [0]:
# Defining the lstm cell for model
def build_lstm(lstm_size, num_layers, batch_size, keep_prob):
    ### Build the LSTM Cell
    
    def build_cell(lstm_size, keep_prob):
        # Use a basic LSTM cell
        lstm = tf.contrib.rnn.BasicLSTMCell(lstm_size)
        
        # Add dropout to the cell
        drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
        return drop
    
    
    # Stack up multiple LSTM layers, for deep learning
    cell = tf.contrib.rnn.MultiRNNCell([build_cell(lstm_size, keep_prob) for _ in range(num_layers)])
    initial_state = cell.zero_state(batch_size, tf.float32)
    
    return cell, initial_state

In [0]:
# Getting the logits
def build_output(lstm_output, in_size, out_size):
    ''' Builds a softmax layer, return the softmax output and logits.
    
        Arguments
        ---------
        
        x: Input tensor
        in_size: Size of the input tensor, for example, size of the LSTM cells
        out_size: Size of this softmax layer
    
    '''

    # Reshape output so it's a bunch of rows, one row for each step for each sequence.
    # That is, the shape should be batch_size*num_steps rows by lstm_size columns
    seq_output = tf.concat(lstm_output, axis=1)
    x = tf.reshape(seq_output, [-1, in_size])
    
    # Connect the RNN outputs to a softmax layer
    with tf.variable_scope('softmax'):
        softmax_w = tf.Variable(tf.truncated_normal((in_size, out_size), stddev=0.1))
        softmax_b = tf.Variable(tf.zeros(out_size))
    
    # Since output is a bunch of rows of RNN cell outputs, logits will be a bunch
    # of rows of logit outputs, one for each step and sequence
    logits = tf.matmul(x, softmax_w) + softmax_b
    
    # Use softmax to get the probabilities for predicted characters
    out = tf.nn.softmax(logits, name='predictions')
    
    return out, logits

In [0]:
# Calculating the loss
def build_loss(logits, targets, lstm_size, num_classes):
    ''' Calculate the loss from the logits and the targets.
    
        Arguments
        ---------
        logits: Logits from final fully connected layer
        targets: Targets for supervised learning
        lstm_size: Number of LSTM hidden units
        num_classes: Number of classes in targets
        
    '''
    
    # One-hot encode targets and reshape to match logits, one row per batch_size per step
    y_one_hot = tf.one_hot(targets, num_classes)
    y_reshaped = tf.reshape(y_one_hot, logits.get_shape())
    
    # Softmax cross entropy loss
    loss = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y_reshaped)
    loss = tf.reduce_mean(loss)
    return loss

In [0]:
# Defining the optimizer for training
def build_optimizer(loss, learning_rate, grad_clip):
    ''' Build optmizer for training, using gradient clipping.
    
        Arguments:
        loss: Network loss
        learning_rate: Learning rate for optimizer
    
    '''
    
    # Optimizer for training, using gradient clipping to control exploding gradients
    tvars = tf.trainable_variables()
    grads, _ = tf.clip_by_global_norm(tf.gradients(loss, tvars), grad_clip)
    train_op = tf.train.AdamOptimizer(learning_rate)
    optimizer = train_op.apply_gradients(zip(grads, tvars))
    
    return optimizer

In [0]:
class CharRNN:
    
    def __init__(self, num_classes, batch_size=64, num_steps=50, 
                       lstm_size=128, num_layers=2, learning_rate=0.001, 
                       grad_clip=5, sampling=False):
    
        # When we're using this network for sampling later, we'll be passing in
        # one character at a time, so providing an option for that
        if sampling == True:
            batch_size, num_steps = 1, 1
        else:
            batch_size, num_steps = batch_size, num_steps

        tf.reset_default_graph()
        
        # Build the input placeholder tensors
        self.inputs, self.targets, self.keep_prob = build_inputs(batch_size, num_steps)

        # Build the LSTM cell
        cell, self.initial_state = build_lstm(lstm_size, num_layers, batch_size, self.keep_prob)

        ### Run the data through the RNN layers
        # First, one-hot encode the input tokens
        x_one_hot = tf.one_hot(self.inputs, num_classes)
        
        # Run each sequence step through the RNN and collect the outputs
        outputs, state = tf.nn.dynamic_rnn(cell, x_one_hot, initial_state=self.initial_state)
        self.final_state = state
        
        # Get softmax predictions and logits
        self.prediction, self.logits = build_output(outputs, lstm_size, num_classes)
        
        # Loss and optimizer (with gradient clipping)
        self.loss = build_loss(self.logits, self.targets, lstm_size, num_classes)
        self.optimizer = build_optimizer(self.loss, learning_rate, grad_clip)

In [0]:
# Hyperparameters
batch_size = 100        # Sequences per batch
num_steps = 100         # Number of sequence steps per batch
lstm_size = 512         # Size of hidden layers in LSTMs
num_layers = 2          # Number of LSTM layers
learning_rate = 0.001   # Learning rate
keep_prob = 0.5         # Dropout keep probability

In [0]:
import time
from collections import namedtuple

In [18]:
epochs = 20
# Print losses every N interations
print_every_n = 50

# Save every N iterations
save_every_n = 200

model = CharRNN(len(vocab), batch_size=batch_size, num_steps=num_steps,
                lstm_size=lstm_size, num_layers=num_layers, 
                learning_rate=learning_rate)

saver = tf.train.Saver(max_to_keep=100)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    # Use the line below to load a checkpoint and resume training
    #saver.restore(sess, 'checkpoints/______.ckpt')
    counter = 0
    for e in range(epochs):
        # Train network
        new_state = sess.run(model.initial_state)
        loss = 0
        for x, y in get_batches(encoded, batch_size, num_steps):
            counter += 1
            start = time.time()
            feed = {model.inputs: x,
                    model.targets: y,
                    model.keep_prob: keep_prob,
                    model.initial_state: new_state}
            batch_loss, new_state, _ = sess.run([model.loss, 
                                                 model.final_state, 
                                                 model.optimizer], 
                                                 feed_dict=feed)
            if (counter % print_every_n == 0):
                end = time.time()
                print('Epoch: {}/{}... '.format(e+1, epochs),
                      'Training Step: {}... '.format(counter),
                      'Training loss: {:.4f}... '.format(batch_loss),
                      '{:.4f} sec/batch'.format((end-start)))
        
            if (counter % save_every_n == 0):
                saver.save(sess, "checkpoints/i{}_l{}.ckpt".format(counter, lstm_size))
    
    saver.save(sess, "checkpoints/i{}_l{}.ckpt".format(counter, lstm_size))

W0701 05:36:59.453284 140694842161024 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

W0701 05:36:59.457752 140694842161024 deprecation.py:323] From <ipython-input-11-6a33ab0f9492>:16: BasicLSTMCell.__init__ (from tensorflow.python.ops.rnn_cell_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
W0701 05:36:59.466363 140694842161024 deprecation.py:323] From <ipython-input-11-6a33ab0f9492>:24: MultiRNNCell.__init__ (from tensorflow.python.ops.rnn_cell_impl) is deprecated and will be removed in a future version.
Instructio

Epoch: 1/20...  Training Step: 50...  Training loss: 3.2945...  0.1537 sec/batch
Epoch: 1/20...  Training Step: 100...  Training loss: 3.1293...  0.1508 sec/batch
Epoch: 1/20...  Training Step: 150...  Training loss: 2.8488...  0.1516 sec/batch
Epoch: 1/20...  Training Step: 200...  Training loss: 2.5176...  0.1540 sec/batch
Epoch: 1/20...  Training Step: 250...  Training loss: 2.4269...  0.1570 sec/batch
Epoch: 1/20...  Training Step: 300...  Training loss: 2.3301...  0.1602 sec/batch
Epoch: 1/20...  Training Step: 350...  Training loss: 2.2631...  0.1561 sec/batch
Epoch: 1/20...  Training Step: 400...  Training loss: 2.1815...  0.1564 sec/batch
Epoch: 1/20...  Training Step: 450...  Training loss: 2.1402...  0.1557 sec/batch
Epoch: 2/20...  Training Step: 500...  Training loss: 2.0739...  0.1557 sec/batch
Epoch: 2/20...  Training Step: 550...  Training loss: 2.0286...  0.1529 sec/batch
Epoch: 2/20...  Training Step: 600...  Training loss: 2.0001...  0.1552 sec/batch
Epoch: 2/20...  T

In [19]:
tf.train.get_checkpoint_state('checkpoints')

model_checkpoint_path: "checkpoints/i9140_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i200_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i400_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i600_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i800_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i1000_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i1200_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i1400_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i1600_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i1800_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i2000_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i2200_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i2400_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i2600_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i2800_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i3000_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i3200_l512.ckpt"
all_model_checkpoint_pa

In [0]:
def pick_top_n(preds, vocab_size, top_n=5):
    p = np.squeeze(preds)
    p[np.argsort(p)[:-top_n]] = 0
    p = p / np.sum(p)
    c = np.random.choice(vocab_size, 1, p=p)[0]
    return c

In [0]:
# generating new text
def sample(checkpoint, n_samples, lstm_size, vocab_size, prime="The "):
    samples = [c for c in prime]
    model = CharRNN(len(vocab), lstm_size=lstm_size, sampling=True)
    saver = tf.train.Saver()
    with tf.Session() as sess:
        saver.restore(sess, checkpoint)
        new_state = sess.run(model.initial_state)
        for c in prime:
            x = np.zeros((1, 1))
            x[0,0] = vocab_to_int[c]
            feed = {model.inputs: x,
                    model.keep_prob: 1.,
                    model.initial_state: new_state}
            preds, new_state = sess.run([model.prediction, model.final_state], 
                                         feed_dict=feed)

        c = pick_top_n(preds, len(vocab))
        samples.append(int_to_vocab[c])

        for i in range(n_samples):
            x[0,0] = c
            feed = {model.inputs: x,
                    model.keep_prob: 1.,
                    model.initial_state: new_state}
            preds, new_state = sess.run([model.prediction, model.final_state], 
                                         feed_dict=feed)

            c = pick_top_n(preds, len(vocab))
            samples.append(int_to_vocab[c])
        
    return ''.join(samples)

In [22]:
tf.train.latest_checkpoint('checkpoints')

'checkpoints/i9140_l512.ckpt'

In [23]:
checkpoint = tf.train.latest_checkpoint('checkpoints')
samp = sample(checkpoint, 2000, lstm_size, len(vocab), prime="Far")
print(samp)

W0701 06:03:52.973279 140694842161024 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/training/saver.py:1276: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for files with this prefix.


Farcester,
And win me that we may be so by hand
That, when you did not, which with a some cause,
To hero it almost the world and whom they'll spire,
Which I shall show, and tempt you to the composition
And that he was but service at this mind,
That, by the wars of all the weaken man,
All me at home with the cust of the corn.

CORNWALL:
Who, sir? he hath a son in the devil,
I should be so, and so, to see my friends:
I will be cholerian, and surely we may,
Being a countenance for the champe of him;
For, and what stand he is imperial seasons?

CASSIUS:
Ay, sir, and show your son of mine own prisoner;
Though he have bought, and take me out of mine.

LUCIO:
And so it, sir; that is the providence of
Her love to see your highness. Would you write
Our charge is at our parting this advised!
What are thy breath at hell? what she were sure
To seek this father, the wind with a sea
Of men and tressage in this state were back'd
To make it so alone.

LEONTES:
Well this, though I should
Am I to tell t

In [24]:
checkpoint = 'checkpoints/i200_l512.ckpt'
samp = sample(checkpoint, 1000, lstm_size, len(vocab), prime="Far")
print(samp)

Fart,
Win sor wont that tor to ther ser aned tor.
Whe the mo tith sotun to tin hh ane beten saslt he se tout oud
At he ar ar tit hat arether.

SAES:
I the the sarde an ator aston and the tor,
Are tise se want oud ant or there sins sou the an tous thee sand ant thil the thou to toud so wote hore wor heat the tererat y wead tor tores,
These te hate wher aled baten ho mins ar thart this and toud shir tathe tor wins the site the te the sarer nall yare, sout ther ther an shal that hor,
An des on hore seat hont werthin than the thou des ise thal ar and ase teus wore to tour sanl int al oud that sethe lot tinthis tine aron sote this the than d tit sherend sot, te antore whot tho sor thes, athal sasdes werle, I al lere wel are and ar tith tous sind ther tin ar artir hite, hh walle te wit tout sisl tire,

han de anteres are an tint and

athe th ir menl oth the ante tot the thal hase al ase he te ale
I and seris sor tas,
I and tis sarl int ore, tar ale to teut hine
Thasl tonu hest oreer four the

In [25]:
checkpoint = 'checkpoints/i600_l512.ckpt'
samp = sample(checkpoint, 1000, lstm_size, len(vocab), prime="Far")
print(samp)

Farles as the pronted
To to the paltares in. Way'd with he thougest
Of then a southers thou hadt and ten to bust
With a with to his and some to she some thourss
Take to to more and thay, this frould that wath,
I that tho goud to the soont ofe menton of the

And to the werting of stale a dourdes thas
Tistorteress the world thou the comnesser the forth,
Weld my liscint, and the wald and me heart tinkeds.

FORSIFLAF:
Thou seelt taks winch our and singladise,
There's thine ant ant shead bet morth mase
Thoure streed of misher to this assers, wored a tist,
I conde steake to stor ou hove of stalise.

PROTOR OLIUS:
Whise he were the wind and sice that that have hang
I than the sore thes thas she whan stile some
To mearsent of an him sonter, as to to hive the porsers
To make is and the wither tay stall' storgh
I spull to stine hun to have are thy mast in sithe
Wich here sto beter thou stick not me worest
And sent of sine and so sour the come to be son
The werth wo straen the soot of he with som